In [2]:
import numpy as np
import pymc as pm
import pytensor.tensor as pt

In [7]:
import numpy as np

# Metalog SPT-parametrized bounded distribution implementation

def check_feasibility(q05, q_alpha, q_1_alpha, alpha, bl, bu):
    # Compute gamma values
    gamma_alpha = (q_alpha - bl) / (bu - q_alpha)
    gamma_1_alpha = (q_1_alpha - bl) / (bu - q_1_alpha)

    # Compute k_alpha
    k_alpha = 0.5 * (1 - 1.66711 * (0.5 - alpha))

    # Compute feasibility bounds
    left = (bl + bu * gamma_alpha**(1 - k_alpha) * gamma_1_alpha**k_alpha) / \
           (1 + gamma_alpha**(1 - k_alpha) * gamma_1_alpha**k_alpha)

    right = (bl + bu * gamma_alpha**k_alpha * gamma_1_alpha**(1 - k_alpha)) / \
            (1 + gamma_alpha**k_alpha * gamma_1_alpha**(1 - k_alpha))
    
    print("left: ", left)
    print("q05: ", q05)
    print("right: ", right)


    return left < q05 < right


def compute_constants(q05, q_alpha, q_1_alpha, alpha, bl, bu):
    # Compute gamma values
    gamma_alpha = (q_alpha - bl) / (bu - q_alpha)
    gamma_05 = (q05 - bl) / (bu - q05)
    gamma_1_alpha = (q_1_alpha - bl) / (bu - q_1_alpha)

    print("gamma_alpha: ", gamma_alpha)
    print("gamma_05: ", gamma_05)
    print("gamma_1_alpha: ", gamma_1_alpha)

    if not check_feasibility(q05, q_alpha, q_1_alpha, alpha, bl, bu):
        raise ValueError("Feasibility condition not satisfied for the given inputs.")

    # Compute constants
    a1 = np.log(gamma_05)
    a2 = 0.5 * (np.log((1 - alpha) / alpha))**(-1) * np.log(gamma_1_alpha / gamma_alpha)
    a3 = ((1 - 2 * alpha) * np.log((1 - alpha) / alpha))**(-1) * \
         np.log((gamma_1_alpha**alpha / gamma_05**2) * (gamma_alpha**(1 - alpha)))

    return a1, a2, a3

def M_y(y, a1, a2, a3):
    return a1 + a2 * np.log(y / (1 - y)) + a3 * (y - 0.5) * np.log(y / (1 - y))

def m_y(y, a2, a3):
    return 1 / (a2 / (y * (1 - y)) + a3 * ((y - 0.5) / (y * (1 - y)) + np.log(y / (1 - y))))

def Mlogit_y(y, bl, bu, a1, a2, a3):
    if y == 0:
        return bl
    elif y == 1:
        return bu
    else:
        My = M_y(y, a1, a2, a3)
        return (bl + bu * np.exp(My)) / (1 + np.exp(My))

def mlogit_y(y, bl, bu, a1, a2, a3):
    if y == 0 or y == 1:
        return 0
    else:
        My = M_y(y, a1, a2, a3)
        my = m_y(y, a2, a3)
        return my * (1 + np.exp(My))**2 / ((bu - bl) * np.exp(My))

In [15]:
#a = calculate_constants(13, 75, 79, bl=0, bu=90)
compute_constants(q05=70, q_alpha=13, q_1_alpha=79, alpha=0.1, bl=0, bu=90)

gamma_alpha:  0.16883116883116883
gamma_05:  3.5
gamma_1_alpha:  7.181818181818182
left:  21.57635368075394
q05:  70
right:  71.42478847648395


(1.252762968495368, 0.8534422659261266, -2.2240222832139036)